In [2]:
# imports pandas
import pandas as pd
import os
import requests

fs_client_id = 'CCHNQLYNIRTT3X151AYFO1LCTZ551GLLDKYWEQPJVEJLOUIV'
fs_client_secret = '2Z04ZC0L2IGSYLLB5YAGJR2YIUH22P1DQNKYNOGINHBJ0WK4'

fsq_api_key = os.environ['FSQ_API_KEY']

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [3]:
# from data import citybikes_paris.csv
citybikes_vancouver = pd.read_csv('../data/citybikes_montreal.csv')
#citybikes_montreal

citybikes_vancouver['name'].count()

227

In [157]:


#query = 'bike'
lati = 48.865983
long = 2.275725
radius = 1000

url = 'https://api.foursquare.com/v3/places/search?ll={},{}&radius={}'.format(lati, long, radius)

headers = {
    "accept": "application/json",
    "Authorization": fsq_api_key
}
response = requests.get(url, headers=headers)
json_data = response.json()

#put data into dataframe, from json_data dictionary first element is results and inside results is a list of dictionaries
# i want to get the name, location, categories, and id
df = pd.DataFrame(json_data['results'])


In [158]:

tips_df = pd.DataFrame()

#get fsq_id and put it in a list
for i in range(len(df)):
    fsq_id = (df['fsq_id'][i])
    url2 = 'https://api.foursquare.com/v3/places/{}/tips'.format(fsq_id)
    response2 = requests.get(url2, headers=headers)
    json_data2 = response2.json()    
    #normalize data and put it in a dataframe df2
    df2 = pd.DataFrame(json_data2)
    #count number of tips and put it in a list
    tips_count = df2['id']

    #append tips_count to tips_df as review_count
    tips_df = tips_df.append({'review_count': len(tips_count)}, ignore_index=True)



C:\Users\milan\AppData\Local\Temp\ipykernel_11808\884184057.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tips_df = tips_df.append({'review_count': len(tips_count)}, ignore_index=True)
C:\Users\milan\AppData\Local\Temp\ipykernel_11808\884184057.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tips_df = tips_df.append({'review_count': len(tips_count)}, ignore_index=True)
C:\Users\milan\AppData\Local\Temp\ipykernel_11808\884184057.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tips_df = tips_df.append({'review_count': len(tips_count)}, ignore_index=True)
C:\Users\milan\AppData\Local\Temp\ipykernel_11808\884184057.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a 

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [159]:
# normalize the columns
location = pd.json_normalize(df['location'])
geocodes = pd.json_normalize(df['geocodes'])
category = pd.json_normalize(df['categories'])
category = pd.json_normalize(category[0])


address = location.formatted_address
lat = geocodes['main.latitude']
long = geocodes['main.longitude']
distance = df['distance']
name = df['name']
category = category['name']

review_count = tips_df['review_count']

site = 'Foursquare'


Put your parsed results into a DataFrame

In [161]:
# create a new dataframe with address, lat, long, distance in meters
bikestation_poi = pd.DataFrame({'category': category, 'name': name, 'address': address, 'lat': lat, 'long': long, 'review_count': review_count, 'site': site})

#select all records from bikestation
bikestation_poi

,category,name,address,lat,long,review_count,site
0,Butcher,Le Bourdonnec au Carré,"172 avenue Victor Hugo, 75016 Paris",48.866120,2.276643,3,Foursquare
1,Bar,Café Lamartine,"182 avenue Victor Hugo (67 rue Spontini), 7511...",48.865940,2.276189,3,Foursquare
2,Garden,Square Lamartine,"3 Square Lamartine, 75116 Paris",48.864819,2.275424,2,Foursquare
3,Café,You Decide,"152 avenue Victor Hugo, 75016 Paris",48.866904,2.278407,6,Foursquare
4,Asian Restaurant,Hansan,"192 avenue Victor Hugo, 75016 Paris",48.865092,2.274127,6,Foursquare
5,Grocery Store / Supermarket,U Express,"14 rue Dufrenoy, 75016 Paris",48.866328,2.273669,1,Foursquare
6,Restaurant,Le Pain Quotidien Victor Hugo,"150 avenue Victor Hugo, 75016 Paris",48.866964,2.278529,10,Foursquare
7,Café,Le Zinc du 16,"58 rue de la Faisanderie, 75016 Paris",48.868135,2.275194,1,Foursquare
8,Bakery,Boulangerie Schou,"96 rue de la Faisanderie (Rue Dufrénoy), 75016...",48.866286,2.273988,3,Foursquare
9,Gastropub,Le Flandrin,"4 place Tattegrain, 75116 Paris",48.864300,2.272098,10,Foursquare


In [4]:
radius = 1000
citybike_foursquare = pd.DataFrame()

for index, row in citybikes_vancouver.iterrows():    
    station_name = row['name']
    lat = row['latitude']
    long = row['longitude']
    
    # build the url
    url = 'https://api.foursquare.com/v3/places/search?ll={},{}&radius={}'.format(lat, long, radius)
    headers = {
    "accept": "application/json",
    "Authorization": fsq_api_key
    }
    response = requests.get(url, headers=headers)
    json_data = response.json()

    df = pd.DataFrame(json_data['results'])

    tips_df = pd.DataFrame()

    #get fsq_id and count reviews
    for i in range(len(df)):
        fsq_id = (df['fsq_id'][i])
        url2 = 'https://api.foursquare.com/v3/places/{}/tips'.format(fsq_id)
        response2 = requests.get(url2, headers=headers)
        json_data2 = response2.json()
        tips_count = 0    
        #normalize data and put it in a dataframe df2
        tips_df = pd.DataFrame(json_data2)
        tips_count = tips_df['id']    

        #append tips_count to tips_df as review_count
        tips_df = tips_df.append({'review_count': len(tips_count)}, ignore_index=True)

    # normalize the columns
    location = pd.json_normalize(df['location'])
    geocodes = pd.json_normalize(df['geocodes'])
    category = pd.json_normalize(df['categories'])
    category = pd.json_normalize(category[0])


    address = location.formatted_address
    lat = geocodes['main.latitude']
    long = geocodes['main.longitude']
    distance = df['distance']
    name = df['name']
    category = category['name']

    review_count = tips_df['review_count']

    # create a new dataframe with address, lat, long, distance in meters
    bikestation_poi = pd.DataFrame({'station':station_name,'category': category, 'name': name, 'address': address, 'lat': lat, 'long': long, 'review_count': review_count, 'site': 'Foursquare'})

    # append to the citybike_foursquare dataframe
    citybike_foursquare = citybike_foursquare.append(bikestation_poi)
        


KeyError: 'id'

In [ ]:
# Create a full dataframe for tips


In [150]:
#citybike_foursquare

# save the dataframe to a csv file
citybike_foursquare.to_csv('../data/citybike_foursquare_vancouver.csv', index=False)

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [37]:
yelp_client_id = 'rt6YwOeyXs5Oqx8HIn-6PQ'
yelp_api_key = os.environ['YELP_API_KEY']


headers = {
    "accept": "application/json",
    "Authorization": f'Bearer {yelp_api_key}'
}



In [40]:
#query = 'bike'
lati = 48.865983
long = 2.275725
radius = 1000

# format this url with latitude and longitude
url = 'https://api.yelp.com/v3/businesses/search?latitude={}&longitude={}&radius={}&sort_by=best_match&limit=10'.format(lati, long, radius)

response = requests.get(url, headers=headers)
json_data = response.json()

# put data into dataframe in tabular format afer normalizing the json data
df = pd.json_normalize(json_data['businesses'])


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,...,coordinates.latitude,coordinates.longitude,location.address1,location.address2,location.address3,location.city,location.zip_code,location.country,location.state,location.display_address
0,WHHt_Jb8Tgidn9mW7oDnIg,la-coïncidence-paris-4,La Coïncidence,https://s3-media2.fl.yelpcdn.com/bphoto/SQZZvL...,False,https://www.yelp.com/biz/la-co%C3%AFncidence-p...,507,"[{'alias': 'french', 'title': 'French'}]",4.5,[],...,48.868105,2.284365,15 rue Mesnil,,,Paris,75116,FR,75,"[15 rue Mesnil, 75116 Paris, France]"


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [110]:
categories = pd.json_normalize(df['categories'])
categories = pd.json_normalize(categories[0])
categories = categories['title']
categories = categories.rename('category')
categories = pd.DataFrame(categories)

name = pd.DataFrame(df['name'])

address = pd.DataFrame(df['location.display_address'])
address = address.rename(columns={'location.display_address':'address'})

lat = pd.DataFrame(df['coordinates.latitude'])
lat = lat.rename(columns={'coordinates.latitude':'lat'})

long = pd.DataFrame(df['coordinates.longitude'])
long = long.rename(columns={'coordinates.longitude':'long'})

rating = pd.DataFrame(df['rating'])
rating = rating.rename(columns={'rating':'rating'})

review_count = pd.DataFrame(df['review_count'])
review_count = review_count.rename(columns={'review_count':'review_count'})

site = "Yelp"

# create a new dataframe with category, name, address, lat, long, rating, review_count, site
yelp = pd.concat([categories, name, address, lat, long, rating, review_count], axis=1)
yelp['site'] = site



,category,name,address,lat,long,rating,review_count,site
0,French,La Coïncidence,"[15 rue Mesnil, 75116 Paris, France]",48.868105,2.284365,4.5,507,Yelp
1,Tea Rooms,Carette,"[4 place du Trocadéro, 75016 Paris, France]",48.863708,2.287207,4.0,246,Yelp
2,Italian,Ozio,"[29 bis rue St Didier, 75016 Paris, France]",48.866310,2.285810,4.5,94,Yelp
3,Bakeries,La Petite Marquise,"[3 place Victor Hugo, 75116 Paris, France]",48.869580,2.285570,4.5,67,Yelp
4,French,Le Poincaré,"[22 avenue Raymond Poincaré, 75116 Paris, France]",48.865057,2.286521,4.0,140,Yelp
5,Museums,Musée Marmottan Monet,"[2 rue Louis Boilly, 75016 Paris, France]",48.859140,2.267550,4.5,69,Yelp
6,International,La Gare,"[19 chaussée de la Muette, 75016 Paris, France]",48.858153,2.272354,3.5,350,Yelp
7,French,Bon,"[25 rue de la Pompe, 75116 Paris, France]",48.860180,2.274980,4.0,119,Yelp
8,French,Le Stella,"[133 avenue Victor Hugo, 75016 Paris, France]",48.867555,2.280705,4.0,75,Yelp
9,Pizza,Mokus l'Ecureuil,"[116 avenue Kléber, 75116 Paris, France]",48.863964,2.287651,4.5,76,Yelp


Put your parsed results into a DataFrame

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating